In [3]:
import os
import logging
from datetime import datetime
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from tqdm import tqdm
import boto3
import zipfile
from io import BytesIO

In [10]:
AWS_ENDPOINT = "http://localhost:9000"
AWS_ACCESS_KEY_ID = "U6dHDkvTv3CdrviA"
AWS_SECRET_ACCESS_KEY = "o8LTVjrVtNuzOv7DjGOxhN0HQDeksEej"
AWS_REGION ="us-east-1"

SPARK_APP_NAME= "fiec-antaq-notebook"
SPARK_MASTER_URL="spark://spark-master:7077"

In [11]:
s3_client = boto3.client('s3',
    endpoint_url=AWS_ENDPOINT,
    aws_access_key_id='U6dHDkvTv3CdrviA',
    aws_secret_access_key='o8LTVjrVtNuzOv7DjGOxhN0HQDeksEej',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='us-east-1'
)
s3_client

In [6]:
def descompactarArquivo(bucket:str,chave:str):
    ano, dataset ,etapa, nomeArquivo = chave.split("/")
    nomeArquivoTXT = nomeArquivo.replace('.zip','.csv')
    chaveArquivoTXT = "/".join([ano,dataset,nomeArquivoTXT])

    zip_obj = s3_client.get_object(Bucket=bucket, Key=chave).get("Body")
    buffer = BytesIO(zip_obj.read())
    
    z = zipfile.ZipFile(buffer)
    for filename in z.namelist():
        file_info = z.getinfo(filename)

        s3_client.upload_fileobj(
            z.open(filename),
            BUCKET_STAGE,
            chaveArquivoTXT
        )
        
        print(f"Arquivo {filename} salvo no bucket stage [{BUCKET_STAGE}/{chaveArquivoTXT}].")

In [12]:
BUCKET_RAW = "antaq-raw"
BUCKET_STAGE = "antaq-stage"
BUCKET_ANALYTICS = "antaq-analytics"

objetosS3 = s3_client.list_objects(Bucket=BUCKET_RAW)

for obj in objetosS3.get("Contents"):
    chave = obj["Key"]
    if "downloaded" in chave:
        descompactarArquivo(BUCKET_RAW,chave)
         # Fazer cópia do arquivo para a pasta processado e apagar da pasta de download
        novaChave = chave.replace("downloaded", "processed")
        
        s3_client.copy_object(
            CopySource={'Bucket': BUCKET_RAW, 'Key': chave},
            Bucket=BUCKET_RAW,
            Key=novaChave)
        
        s3_client.delete_object(Bucket=BUCKET_RAW, Key=chave)
        
        print(f"Arquivo {chave} movido para a pasta de processados.")
    else:
        print(chave)

year=2010/atracacao/processed/2010atracacao.zip
year=2010/carga/processed/2010carga.zip
year=2011/atracacao/processed/2011atracacao.zip
year=2011/carga/processed/2011carga.zip
year=2012/atracacao/processed/2012atracacao.zip
year=2012/carga/processed/2012carga.zip
year=2013/atracacao/processed/2013atracacao.zip
year=2013/carga/processed/2013carga.zip
year=2014/atracacao/processed/2014atracacao.zip
year=2014/carga/processed/2014carga.zip
year=2015/atracacao/processed/2015atracacao.zip
year=2015/carga/processed/2015carga.zip
year=2016/atracacao/processed/2016atracacao.zip
year=2016/carga/processed/2016carga.zip
year=2017/atracacao/processed/2017atracacao.zip
year=2017/carga/processed/2017carga.zip
year=2018/atracacao/processed/2018atracacao.zip
year=2018/carga/processed/2018carga.zip
year=2019/atracacao/processed/2019atracacao.zip
year=2019/carga/processed/2019carga.zip
year=2020/atracacao/processed/2020atracacao.zip
year=2020/carga/processed/2020carga.zip
year=2021/atracacao/processed/20